In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD')
print(os.getcwd())
import pandas as pd
import numpy as np
import datetime
import plotly.express as px

/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD


In [ ]:
##Data prepping
df=pd.read_csv('store_sales_data_processed.csv')

df['date']=pd.to_datetime(df['date'])
df=df.rename(columns={'CLUSTER':'cluster'})
df=df.sort_values(['store_nbr','date'],ascending=[True,True])

##dummies for store type
df.reset_index(drop=True,inplace=True)
store_type_dummies=pd.get_dummies(df.store_type,prefix='st',drop_first=True)
store_type_dummies
df=pd.concat([df,store_type_dummies],axis=1)


##Treating stores that opened later
nonzero_sales=df[['store_nbr','sales','date']].sort_values(['store_nbr','date'],ascending=[True,True])
nonzero_sales=nonzero_sales[nonzero_sales['sales']>0]
nonzero_sales_min_dt=nonzero_sales.groupby(['store_nbr'])['date'].min().reset_index().rename(columns={'date':'min_non_zero_date'})
nonzero_sales_min_dt['starting_dt']=df['date'].min()
nonzero_sales_min_dt['days_from_start']=(nonzero_sales_min_dt['min_non_zero_date']-nonzero_sales_min_dt['starting_dt'])/np.timedelta64(1, 'D')

later_opened_stores=nonzero_sales_min_dt[nonzero_sales_min_dt['days_from_start']>1]


##joining to main dataset

df1=pd.merge(df,later_opened_stores,on=['store_nbr'],how='left')
df1['starting_dt']=df.date.min()
df1.loc[(df1['min_non_zero_date'].isnull()),'min_non_zero_date']=df1['starting_dt']
df1['active']='Y'
df1.loc[(df1['date']<df1['min_non_zero_date']),'active']='N'

###Transforming time dimension
timevar=df1[['year','month','daynum_of_week']].drop_duplicates(keep='first').reset_index(drop=True)

time_yr=pd.get_dummies(timevar.year,prefix='yr')

timevar=pd.concat([timevar,time_yr],axis=1)

timevar['sin_mnth']=np.sin(np.deg2rad(df.month))

timevar['cos_mnth']=np.cos(np.deg2rad(df.month))

timevar['sin_dow']=np.sin(np.deg2rad(df.daynum_of_week))

timevar['cos_dow']=np.cos(np.deg2rad(df.daynum_of_week))


###




df1=pd.merge(df1,timevar,on=['year','month','daynum_of_week'],how='inner')
df1=df1[df1['active']=='Y']
df1.reset_index(drop=True,inplace=True)

hdf=df1[~df1['holiday_type'].isnull()]
hdf_select=hdf[['date','store_nbr','holiday_type','transferred']]

criteria1=hdf_select['holiday_type']=='Holiday'
criteria2=hdf_select['transferred']==True

c1=criteria1 & criteria2

##Dealing with holidays

hdf_select['hol_flg']=1

hdf_select.loc[c1,'hol_flg']=0

criteria3=hdf_select['holiday_type']=='Work Day'

hdf_select.loc[criteria3,'hol_flg']=0

hdf_final=hdf_select[['date','store_nbr','hol_flg']]

df1=pd.merge(df1,hdf_final,on=['date','store_nbr'],how='left')

df1.loc[(df1['hol_flg'].isnull()),'hol_flg']=0


df1.reset_index(drop=True,inplace=True)


###trig transform store_nbr

df1['str_nbr_sin']=np.sin(np.deg2rad(df1.store_nbr))
df1['str_nbr_cos']=np.cos(np.deg2rad(df1.store_nbr))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
cluster_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [ ]:
###Constructing the modeling loop

from sklearn.ensemble import IsolationForest


cluster_list=list(set(df1['cluster'].to_list()))

final_df=pd.DataFrame()

for i in range(10,18):
  cluster_df=df1[df1['cluster']==i].reset_index(drop=True)

  model_data=cluster_df[['sales','promotion_count','month','sin_mnth','cos_mnth','sin_dow','cos_dow','st_B','st_C','st_D','st_E','hol_flg','str_nbr_sin','str_nbr_cos']]
  
  if_model=IsolationForest(n_estimators=200,max_samples=0.8,max_features=14,contamination=0.05,random_state=42)
  if_model.fit(model_data)

  cluster_df['outliers']=pd.Series(if_model.predict(model_data)).apply(lambda x: 'Y' if (x==-1) else 'N' )
  score=if_model.decision_function(model_data)
  cluster_df['scores']=score

  cluster_df.loc[(cluster_df['month']==1) & (cluster_df['day_of_mnth']==1),'outliers']='N'

  
  final_df=final_df.append(cluster_df)
  
  fig=px.scatter(cluster_df, x="date", y="sales", color='outliers', facet_row="store_nbr",color_discrete_map={'Y':'red','N':'blue'})

  title='cluster '+str(i)

  fig.update_layout(
    title={
        'text': title,
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

  fig.show()
  print('\n')
  print(if_model.get_params)
  print('\n')


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names





<bound method BaseEstimator.get_params of IsolationForest(contamination=0.05, max_features=14, max_samples=0.8,
                n_estimators=200, random_state=42)>




/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names





<bound method BaseEstimator.get_params of IsolationForest(contamination=0.05, max_features=14, max_samples=0.8,
                n_estimators=200, random_state=42)>




/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



...



<bound method BaseEstimator.get_params of IsolationForest(contamination=0.05, max_features=14, max_samples=0.8,
                n_estimators=200, random_state=42)>




/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names





<bound method BaseEstimator.get_params of IsolationForest(contamination=0.05, max_features=14, max_samples=0.8,
                n_estimators=200, random_state=42)>




/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names





<bound method BaseEstimator.get_params of IsolationForest(contamination=0.05, max_features=14, max_samples=0.8,
                n_estimators=200, random_state=42)>




In [ ]:
final_df.to_csv('modeled_data.csv',index=False)